# App Wetter einspielen

nachfolgend wird App "Wetter einspielen" skiziziert:  
  
**Ziele/Anforderungen**
- regelmäßige Abfrage der Daten von https://dbup2date.uni-bayreuth.de/blocklysql/wetterdaten.html
- Integration der Daten in eine SQL-DB.  
- Übersetzung von Koordinaten in Postleitzahlen

## Abfrage der Website und Link extrahieren

Finde auf Website den Link auf aktulle Zip-Datei mit Wetterdaten.

In [ ]:
from bs4 import BeautifulSoup
import requests


url = "https://dbup2date.uni-bayreuth.de/blocklysql/"
html_file = "wetterdaten.html"
resp = requests.get(url+html_file)


soup = BeautifulSoup(resp.content)

for link in soup.find_all('a', href=True):
    if "CSV" in link['href']:
        path_zip_file_name = link['href']
        print( path_zip_file_name)
        break

        
# https://docs.python.org/3/library/pathlib.html
from pathlib import Path

path = Path(path_zip_file_name)
zip_file_name = path.name
print(zip_file_name)

downloads/wetterdaten/2019-07-02_wetterdaten_CSV.zip
2019-07-02_wetterdaten_CSV.zip


## Lade und speichere Zip-Datei mit Wetterdaten

In [26]:
import requests
import re

def get_filename_from_cd(cd):
    """
    Get filename from remote
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]


# aktulle Wetterdaten in Zip laden
# url = 'https://dbup2date.uni-bayreuth.de/blocklysql/downloads/wetterdaten/2019-06-21_wetterdaten.sql'
r = requests.get(url + path_zip_file_name, allow_redirects=True)
filename = get_filename_from_cd(r.headers.get('content-disposition'))


# Datei lokal speichern
saved_pathfile =  Path.cwd().joinpath("data").joinpath(zip_file_name)
open(saved_pathfile, 'wb').write(r.content)
saved_pathfile

WindowsPath('F:/Python/_Working/Wetter-Gustini/data/2019-07-02_wetterdaten_CSV.zip')

# Einspielen der Daten in SQL-DB
Weitere Anforderungen:  
- Intgration lediglich neuer Daten (anhand Datum) -> SQL Merge oder filtern nach Datum?
- Übersetzung von Koordinaten in Postleitzahl
- Anreichen der Koordinatendaten mit Postleitzahl

    - statische Übersetzung -> einmaliges Mapping von Koordinaten oder 
    - dynamische Übersetzung -> Loopup der Koordinaten in geomaps per Postleitzahl?
    - -> vorher alle Koordinaten abfragen und sehen, ob es eine PLZ gibt, wenn nicht abbrechen und User um manuelles mapping in Csv bitten 

## Importiere CSV (Wetterstationen) in DataFrame (data_stationen)

**Codeschnipsel: csv aus zipfile in DataFrame imporieren**

import pandas as pd  
import zipfile  
zf = zipfile.ZipFile('C:/Users/Analytics Vidhya/Desktop/test.zip')   
df = pd.read_csv(zf.open(zipfile.ZipFile.namelist(zf)[0]))   


In [27]:
import pandas as pd
import zipfile


file_name = 'wetterdaten_Wetterstation.csv'

# ensure utf-encoding, ensure delimiter ";"
with zipfile.ZipFile(saved_pathfile) as myzip:
    with myzip.open(file_name) as myfile:
        data_stationen = pd.read_csv(myfile,\
        delimiter = ";",\
        index_col=False,\
        header=0,\
        decimal=",",\
        encoding='cp1250') # convert typical german encoding


del data_stationen["Unnamed: 6"]
data_stationen.head(10)

,S_ID,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber
0,102,Leuchtturm Alte Weser,53.863,8.127,32,DWD
1,164,Angermünde,53.032,13.991,54,DWD
2,183,Arkona,54.679,13.434,42,DWD
3,232,Augsburg,48.425,10.942,461,DWD
4,282,Bamberg,49.874,10.921,240,DWD
5,403,Berlin-Dahlem (FU),52.454,13.302,51,DWD
6,430,Berlin-Tegel,52.564,13.309,36,DWD
7,433,Berlin-Tempelhof,52.468,13.402,48,DWD
8,691,Bremen,53.045,8.798,4,DWD
9,722,Brocken,51.799,10.618,1134,DWD


## Importiere Lageangaben für Postleitzahlen

Lese CSV (Geodaten importiert) in DataFrame (mappping_zipcode_coordinate)

In [28]:
import pandas as pd
import numpy as np


file_name = 'data/geodaten_de.csv'
new_columns = ['Plz', 'Ort', 'Latitude', 'Longitude']
dtypes: {'Plz':str, 'Ort':str, 'Latitude':float, 'Longitude':float}


def cast_to_str(x):
    if len(x) >4:
        return str(x)
    else:
        return "0" +str(x)
        

# ensure utf-encoding, ensure delimiter ";"
mappping_zipcode_coordinate = pd.read_csv(file_name,\
    delimiter=";",\
    index_col=False,\
    header=0,
    names=new_columns,
    dtype = {'Plz': np.str, 'Ort': str, 'Latitude':float , 'Longitude': float},\
    encoding='cp1250' # convertes typical german encoding
    #converters = {"Plz": cast_to_str}
    )


mappping_zipcode_coordinate.head(10)

,Plz,Ort,Latitude,Longitude
0,01067,Dresden,51.0600,13.7211
1,01069,Dresden,51.0396,13.7389
2,01097,Dresden,51.0667,13.7440
3,01099,Dresden,51.0926,13.8290
4,01109,Dresden,51.1201,13.7620
5,01127,Dresden,51.0796,13.7333
6,01129,Dresden,51.0968,13.7274
7,01139,Dresden,51.0806,13.6911
8,01157,Dresden,51.0643,13.6708
9,01159,Dresden,51.0426,13.7007


## Finde nächstgelegene Postleitzahl
Finde für jede Wetterstation (data_stationen), finde die nächstgelegene Postleitzahl (mappping_zipcode_coordinate) anhand der kleinsten Distanz von Koordinaten

#### function map_nearest_zipcode

neue Datenstruktur->

1.long und lat übernehmen

2. kopie des mapping-dataframe  + neue spalten mit kritische koordinatenwerten anfügen

3. mittlere quadratische abstand berechnen: differenz, quaddieren, addieren-> jeweils punktweiser abstand

4. zeilen mit nächstgelegene koordinaten extrahieren:  minimum aller abständd  -> index extrahieren - > gesamte zeile zurügeben


Erweiere DataFrame(Stationen) um leere Felder

In [29]:
appended_columns = mappping_zipcode_coordinate.columns + "_matched"

data_stationen.loc[:, str(appended_columns[0])] = np.nan
data_stationen.loc[:, str(appended_columns[1])] = np.nan
data_stationen.loc[:, str(appended_columns[2])] = np.nan
data_stationen.loc[:, str(appended_columns[3])] = np.nan

data_stationen.head(10)

,S_ID,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber,Plz_matched,Ort_matched,Latitude_matched,Longitude_matched
0,102,Leuchtturm Alte Weser,53.863,8.127,32,DWD,NaN,NaN,NaN,NaN
1,164,Angermünde,53.032,13.991,54,DWD,NaN,NaN,NaN,NaN
2,183,Arkona,54.679,13.434,42,DWD,NaN,NaN,NaN,NaN
3,232,Augsburg,48.425,10.942,461,DWD,NaN,NaN,NaN,NaN
4,282,Bamberg,49.874,10.921,240,DWD,NaN,NaN,NaN,NaN
5,403,Berlin-Dahlem (FU),52.454,13.302,51,DWD,NaN,NaN,NaN,NaN
6,430,Berlin-Tegel,52.564,13.309,36,DWD,NaN,NaN,NaN,NaN
7,433,Berlin-Tempelhof,52.468,13.402,48,DWD,NaN,NaN,NaN,NaN
8,691,Bremen,53.045,8.798,4,DWD,NaN,NaN,NaN,NaN
9,722,Brocken,51.799,10.618,1134,DWD,NaN,NaN,NaN,NaN


Extrahiere Koordination aller Postleitzahlen zur schnellen Abfrage

In [30]:
lat_values = mappping_zipcode_coordinate["Latitude"].values
long_values = mappping_zipcode_coordinate["Longitude"].values

Beschreibe Suche der nächstgelegenen Postleitzahl

In [31]:
def find_nearest_zipcode(row):
    
    # a row as a data_station
    para_lat = row["Geo_Breite"]
    para_long = row["Geo_Laenge"]
    
    # use arrays of coordinates and subtract individual coordinate
    lat = lat_values -para_lat
    long = long_values -para_long
    
    # calc distance each to individual coordinate and find minimal distance as nearest location
    distance =  abs(long) + abs(lat) 
    nearest_location = mappping_zipcode_coordinate.iloc[np.argmin(distance)]

    return nearest_location

Wende Funktion zur Suche nächstgelegener Postleitzahl für alle Wetterstationen an und speichere diese Angaben zur Wetterstation

In [32]:
data_stationen[["Plz_matched", "Ort_matched", "Longitude_matched", "Latitude_matched"]] \
    = data_stationen.apply(find_nearest_zipcode, axis=1)
data_stationen

,S_ID,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber,Plz_matched,Ort_matched,Latitude_matched,Longitude_matched
0,102,Leuchtturm Alte Weser,53.863,8.127,32,DWD,26486,Nordseebad Wangerooge,7.91141,53.7896
1,164,Angermünde,53.032,13.991,54,DWD,16278,Angermünde,14.01040,53.0547
2,183,Arkona,54.679,13.434,42,DWD,18556,Dranske,13.31760,54.6303
3,232,Augsburg,48.425,10.942,461,DWD,86169,Augsburg,10.90450,48.4209
4,282,Bamberg,49.874,10.921,240,DWD,96050,Bamberg,10.93140,49.8809
5,403,Berlin-Dahlem (FU),52.454,13.302,51,DWD,12203,Berlin,13.31300,52.4464
6,430,Berlin-Tegel,52.564,13.309,36,DWD,13405,Berlin,13.30030,52.5607
7,433,Berlin-Tempelhof,52.468,13.402,48,DWD,12099,Berlin,13.40550,52.4618
8,691,Bremen,53.045,8.798,4,DWD,28201,Bremen,8.80970,53.0569
9,722,Brocken,51.799,10.618,1134,DWD,38879,Schierke,10.62960,51.7873


In [33]:
data_stationen.to_pickle("data/stat.pkl")

In [34]:
# https://docs.python.org/3/library/pathlib.html
from pathlib import Path


saved_pathfile = None
#path = Path(path_zip_file_name)
#zip_file_name = path.name
if not saved_pathfile:
    saved_pathfile =  Path.cwd().joinpath("data") #.joinpath(zip_file_name)
    saved_pathfile = sorted(saved_pathfile.glob('????-??-??_wetterdaten_CSV.zip'), reverse=True)[0]


saved_pathfile

WindowsPath('F:/Python/_Working/Wetter-Gustini/data/2019-07-02_wetterdaten_CSV.zip')

In [35]:
import pandas as pd
import zipfile


file_name = 'wetterdaten_Wettermessung.csv'

# ensure utf-encoding, ensure delimiter ";"
with zipfile.ZipFile(saved_pathfile) as myzip:
    with myzip.open(file_name) as myfile:
        data_wetter = pd.read_csv(myfile,\
        parse_dates=["Datum"],\
        delimiter = ";",\
        index_col=False,\
        header=0,\
        decimal=",",\
        encoding='cp1250') # convert typical german encoding


del data_wetter["Unnamed: 14"]
data_wetter.head()

,Stations_ID,Datum,Qualitaet,Min_5cm,Min_2m,Mittel_2m,Max_2m,Relative_Feuchte,Mittel_Windstaerke,Max_Windgeschwindigkeit,Sonnenscheindauer,Mittel_Bedeckungsgrad,Niederschlagshoehe,Mittel_Luftdruck
0,102,2017-12-27,3.0,NaN,4.0,5.0,5.7,87.63,9.7,16.3,0.000,NaN,NaN,981.17
1,102,2017-12-28,3.0,NaN,2.7,4.7,6.3,83.42,12.6,20.9,1.900,NaN,NaN,987.32
2,102,2017-12-29,3.0,NaN,1.8,3.0,4.5,86.83,13.2,20.5,4.533,NaN,NaN,994.75
3,102,2017-12-30,3.0,NaN,2.8,5.3,7.9,92.00,13.8,25.0,0.000,NaN,NaN,989.25
4,102,2017-12-31,3.0,NaN,6.1,8.3,11.0,93.38,11.4,20.1,0.000,NaN,NaN,991.57


In [36]:
data_wetter.describe()

,Stations_ID,Qualitaet,Min_5cm,Min_2m,Mittel_2m,Max_2m,Relative_Feuchte,Mittel_Windstaerke,Max_Windgeschwindigkeit,Sonnenscheindauer,Mittel_Bedeckungsgrad,Niederschlagshoehe,Mittel_Luftdruck
count,41357.000000,41269.000000,37796.000000,40711.000000,41294.000000,40711.000000,41082.000000,39682.000000,39727.000000,40882.000000,38260.000000,39047.000000,41143.000000
mean,2986.434558,2.891783,3.435110,5.245457,9.496433,13.797841,74.927957,4.298800,11.407267,5.475740,5.339783,1.834546,977.367351
std,2257.453778,0.452468,6.598906,6.927861,7.962588,9.473841,13.773502,2.758013,5.042579,4.834078,2.336604,4.822293,51.115767
min,102.000000,1.000000,-24.300000,-30.500000,-27.800000,-26.300000,7.000000,0.300000,1.700000,0.000000,0.000000,0.000000,683.620000
25%,1358.000000,3.000000,-1.100000,0.400000,3.600000,6.300000,65.380000,2.400000,7.900000,0.483000,3.800000,0.000000,964.440000
50%,2712.000000,3.000000,3.000000,5.000000,9.000000,13.200000,76.170000,3.500000,10.400000,4.741500,6.000000,0.000000,993.200000
75%,4271.000000,3.000000,8.400000,10.700000,16.000000,21.400000,85.580000,5.300000,13.700000,9.583000,7.400000,1.400000,1009.350000
max,15000.000000,3.000000,21.900000,23.900000,31.100000,38.300000,100.000000,25.600000,60.400000,16.700000,8.000000,105.300000,1042.350000


In [37]:
data_wetter.dtypes

Stations_ID                         int64
Datum                      datetime64[ns]
Qualitaet                         float64
Min_5cm                           float64
Min_2m                            float64
Mittel_2m                         float64
Max_2m                            float64
Relative_Feuchte                  float64
Mittel_Windstaerke                float64
Max_Windgeschwindigkeit           float64
Sonnenscheindauer                 float64
Mittel_Bedeckungsgrad             float64
Niederschlagshoehe                float64
Mittel_Luftdruck                  float64
dtype: object

In [38]:
data_wetter.to_pickle("data/wett.pkl")

# Code Snippets

In [ ]:
## OLD

import zipfile

with zipfile.ZipFile(save_pathfile) as myzip:
    with myzip.open('wetterdaten_Wetterstation.csv') as myfile:
        print(myfile.read())

### Importiere CSV (Wetterstationen) zu List of Lists

In [ ]:
## OLD
file_name = "data/Beispiel_wetterdaten_Wetterstation.csv"

from csv import reader
list_data = list(reader(open(file_name)))
print (list_data[:3])


Importiere CSV (Wetterstationen) in DataFrame

In [1]:
## OLD
import pandas as pd

file_name = "data/Beispiel_wetterdaten_Wetterstation.csv"

# ensure utf-encoding, ensure delimiter ";"
data_stationen = pd.read_csv(file_name,\
    delimiter = ";",\
    index_col=0,\
    header=0,\
    decimal=",",\
    encoding='cp1250') # convertes typical german encoding
del data_stationen["Unnamed: 6"]
data_stationen

,Standort,Geo_Breite,Geo_Laenge,Hoehe,Betreiber
S_ID,,,,,
102,Leuchtturm Alte Weser,53.863,8.127,32,DWD
164,Angermünde,53.032,13.991,54,DWD
183,Arkona,54.679,13.434,42,DWD
232,Augsburg,48.425,10.942,461,DWD
282,Bamberg,49.874,10.921,240,DWD
403,Berlin-Dahlem (FU),52.454,13.302,51,DWD
430,Berlin-Tegel,52.564,13.309,36,DWD
433,Berlin-Tempelhof,52.468,13.402,48,DWD
691,Bremen,53.045,8.798,4,DWD


### Aufbereitung des ursprünglichen CSV (Geodaten)
Erstelle neue Liste mit Zeilen (list_data)

In [ ]:
from csv import writer


# http://www.csv-geodaten.de/geokoordinaten_deutschland.php
file_name = "data/geodaten_UTF8-CRLF.csv"
list_data = []
temp_row = []


'''
EXAMPLE: -> adjust at "^"
['"Plz^', 'Ort', 'Longitude', 'Latitude^"\n']
['"01067^', '"Dresden"', '51.06', '13.7211^"\n']
['"01069^', '"Dresden"', '51.0396', '13.7389^"\n']
['"01097^', '"Dresden"', '51.0667', '13.744^"\n']
'''        


with open(file_name) as fid:
    for row in fid.readlines():
        temp_row = row.split(";")
        
        # correct specific row: ['"73566', '"Bartholom?48.7457', '9.96309"\n']
        if temp_row[0] == '"73566':
            temporal_string = temp_row[1].split("?")
            print("\n===")
            print(temp_row)
            print(temporal_string)
            temporal_string[0] = temporal_string[0]+'"'
            temp_row = [temp_row[0], temporal_string[0], temporal_string[1], temp_row[-1]] # create no row as list-type
            print (temp_row)
            print("===")

        simple_error_cases = ('"19294', '"25863' ,  '"25869',  '"29345',  '"38557',  '"52391',  '"53508',  '"53560',  '"72184',  '"86356',  '"87773',  '"88239',  '"88299',  '"88316',  '"88400',  '"88441',  '"91347',  '"92648',  '"92685',  '"18374',  '"18375',  '"87509',  '"87538',  '"87545',  '"88161',  '"87545',  '"87545')
        
        if temp_row[0] in simple_error_cases:
        
            temporal_string = temp_row[1].replace('?','"')
            print("\n===")
            print(temp_row)
            print(temporal_string)
            #temporal_string[0] = temporal_string[0]+'"'
            temp_row = [temp_row[0], temporal_string, temp_row[2], temp_row[3]]
            print (temp_row)
            print("===")
            
        
        # add " to the end of first element
        temp_row[0] = temp_row[0]+'"'
        
        # remove "\n at the end of forth element
        last_string_element = temp_row[3].strip('"\n')
        temp_row.pop(3)
        temp_row.append(last_string_element)

        # build new list
        list_data.append(temp_row)

        
print("\n\n")        
print(list_data[:4])

### Schreibe Liste mit Zeilen (list_data) in neue CSV (Geodaten formatiert)

In [ ]:
import csv

file_name = 'data/geodata_formatted_Utf8_CLF.csv'
with open(file_name, mode='w') as fid:
    csv_writer = csv.writer(fid,\
        delimiter=';',\
        quotechar="'",\
        quoting=csv.QUOTE_NONE,\
        lineterminator='\n' # for WINDOWS
    )
    for elem in list_data:
        csv_writer.writerow(elem)